# Imports

In [ ]:
# imports

import pandas as pd
import os, glob
from os import path
import shutil
import numpy as np
from IPython.display import Image

# Funktionen und Befehle zur Zusammenfassung der einzelnen Dateien

## Funktionen

In [ ]:
def create_folders(src, csv_stat_type):
  """
    Creates subfolders for the csv files.

            Parameters:
                    src (str)           : Destination for the folders
                    csv_stat_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    None
  """

  os.mkdir(os.path.join(src, "1_{folder_type}".format(folder_type=csv_stat_type)))

In [ ]:
def attach_ID(src, start=100):
  """
    Attaches a specific ID to the name of every file,
    all files in a folder get the same ID.

            Parameters:
                    src (str)   : Path/source of the folders
                    start (int) : Starting point for the IDs (default 100)

            Returns:
                    None
  """

  id = start
  for subdir, dirs, files in sorted(os.walk(src)):
    id += 1
    for filename in files:
      filepath = subdir + os.sep + filename
      new_filename = str(id) + "_" + filename
      new_filepath = subdir + os.sep + new_filename
      os.rename(filepath, new_filepath)

In [ ]:
def move_to_folder(src, dst, name):
  """
    Move the files in each matchday folder to the one with the right topic.

            Parameters:
                    src (str)  : Path/source of the folders
                    dst (str)  : Destination for the files 
                    name (str) : Name of the statistic type, e.g. summary or passing

            Returns:
                    None
  """

  for subdir, dir, files in sorted(os.walk(src)):
    for file in files:
      filepath = subdir + os.sep + file
      if file.endswith("_{}.csv".format(name)):
        shutil.copy(path.join(src, filepath), dst)
        

In [ ]:
def move_first_file(src, csv_type):
  """
    Moves the first file from current subfolder to the preceding folder.

            Parameters:
                    src (str)      : Path/source of the folders
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    None
  """

  src_before = src + "/1_{}".format(csv_type)
  shutil.move(os.path.join(src_before, os.listdir(src_before)[0]), src)

In [ ]:
def remove_first_row(src, csv_type):
  """
    Removes the first row of every csv file.

            Parameters:
                    src (str)      : Path/source of the folders
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    None
  """
  
  os.chdir(src + "/1_{}/".format(csv_type))
  allFiles = glob.glob("*.csv")
  for file in allFiles:
    df = pd.read_csv(file)
    df = df.iloc[1:,]
    df.to_csv(file)
    print(f"{file} has removed row 0")

In [ ]:
def put_first_file_back(src, csv_type):
  """
    Moves the first file back to the correct subfolder.

            Parameters:
                    src (str)      : Path/source of the folders
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    None
  """
  
  src_before = src + "/1_{}".format(csv_type)
  [shutil.move(src + "/{}".format(file), src_before) for file in os.listdir(source) if file.endswith("_{}.csv".format(csv_type))]

In [ ]:
def join_csv_files(src, csv_type):
  """
    Imports the csv files from the folder.

            Parameters:
                    src (str)      : Path/source of the folders
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    None
  """

  src_before = src + "/1_{}".format(csv_type)
  allFiles = glob.glob(src_before + "/*.csv")
  allFiles.sort()
  with open(src_before + '.csv', 'wb') as outfile:
    for i, fname in enumerate(allFiles):
      with open(fname, 'rb') as infile:
        if i != 0:
          infile.readline()
        shutil.copyfileobj(infile, outfile)
        print(fname + " has been imported.")

In [ ]:
def extract_team_names(src, csv_type):
  """
    Extracts the team names in the right order.

            Parameters:
                    src (str)      : Path/source of the folders
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing

            Returns:
                    teamnames_order (list) : List of the teamnames in order of the IDs
  """

  teamnames_order = []
  files = os.listdir(src + "/1_{}".format(csv_type))

  for file in files:
    teamnames_order.append(file)

  teamnames_order = sorted(teamnames_order)
  if csv_type == "def":
    teamnames_order = [teamname[6:-8] if len(teamname) == 17 else teamname[4:-8] for teamname in teamnames_order]
  elif csv_type == "poss":
    teamnames_order = [teamname[6:-9] if len(teamname) == 18 else teamname[4:-9] for teamname in teamnames_order]
  elif csv_type == "passing":
    teamnames_order = [teamname[6:-12] if len(teamname) == 21 else teamname[4:-12] for teamname in teamnames_order]
  elif csv_type == "passtypes":
    teamnames_order = [teamname[6:-14] if len(teamname) == 23 else teamname[4:-14] for teamname in teamnames_order]
  elif csv_type == "misc":
    teamnames_order = [teamname[6:-9] if len(teamname) == 18 else teamname[4:-9] for teamname in teamnames_order]
  elif csv_type == "summary":
    teamnames_order = [teamname[6:-12] if len(teamname) == 21 else teamname[4:-12] for teamname in teamnames_order]
  else:
    return
  

  return teamnames_order

In [ ]:
def team_names_and_gameID(csv_file, csv_type, teamn_o):
  """
    Puts the right team name to the players in the data frame,
    puts in the correct game IDs.

            Parameters:
                    csv_file (str) : Name of the csv file where the team names are added
                    csv_type (str) : Statistic type of the csv file, e.g. summary or passing
                    teamn_o (list) : List of the teamname order

            Returns:
                    None
  """

  df = pd.read_csv(csv_file, header=1)
  team = -1
  teamnames_for_df = []
  if csv_type != "shots":
    for value in df["ID"]:
      if value == 1:
        team += 1
        teamnames_for_df.append(teamn_o[team])
      else:
        teamnames_for_df.append(teamn_o[team])

    df["Team"] = teamnames_for_df


  gameID = 0
  counter = 0
  id_list = []

  if csv_type != "shots":
    for values in df["ID"]:
      if values == 1:
        if counter % 2 == 0:
          gameID +=1
          counter += 1
          id_list.append(gameID)
        else:
          counter += 1
          id_list.append(gameID)
      else:
        id_list.append(gameID)
  else:
    for values in df["ID"]:
      if values == 1:
        gameID += 1
        id_list.append(gameID)
      else:
        id_list.append(gameID)
  df["GameID"] = id_list
  df.to_csv(source + "/{name}_final.csv".format(name=csv_type))

## Ausführung der Funktionen

This whole "Ausführung der Funktionen" block is to be executed completely four times, once for each season and league


In [ ]:
# source in which the operations should take place
source = "/content/drive/MyDrive/BA/BA Daten Original/Saison 17.18/BL"
csv_stat_types = ["def", "misc", "passing", "passtypes", "poss", "shots", "summary"]

# create a folder for every stat type
for types in csv_stat_types:
  create_folders(source, types)

In [ ]:
# attach an ID to every csv file in the folder structure of the source
attach_ID(source)

In [ ]:
%%time

# move every csv file in the folder structure to the right stat type folder
# caution: It is advised to start with at most two stat types, e.g. "def" and "misc", to see how long it will take and choose, 
#          based on that time, how much stat type csv files it should move at once. This process took up to 10 minutes for one stat type in Google Colab.

names_list_move = ["def"] # done:    ; not yet done: "def", "misc", "passing", "passtypes", "poss", "summary", "shots"

for name in names_list_move:
  move_to_folder(source, source + "/1_{}".format(name), name)

In [ ]:
# extract teamnames for every file and check if all are present

for name in ["def"]: 
  teamnames_order = extract_team_names(source, name)
  if len(np.unique(teamnames_order)) == 18: # == 20 for Primera División
    print(True)

In [ ]:
print(np.unique(teamnames_order))

In [ ]:
# merge the csv files of each folder 
# caution: It is advised to go through the stat types one by one, as the system may not be able to handle multiple stat types at once 
#          correctly and may bring out wrong files

names_list_join = ["def"]  # done:    ; not yet done: "def", "misc", "passing", "passtypes", "poss", "summary", "shots" 

for name in names_list_join:
  move_first_file(source, name)
  remove_first_row(source, name)
  put_first_file_back(source, name)
  join_csv_files(source, name)

Check the csv files and update as follows:


*   open the file, e.g. in Excel
*   move all rows from the second one downwards by one
*   search for "Player" (/"Minute" for the shots file) with the search function
*   move the row starting with "Player" (/"Minute" for the shots file) to the second position, where the space is empty after moving the rows downwards by one
*   add "ID," to the front of the "Player" row (/"Minute" row for the shots file)
*   move the rows that where under the original position of the "Player" row (/"Minute" row for the shots file) up by one
*   add the ID in front of every of those just moved rows
*   extract the row that is merged with the last row of those just moved rows and put that in the empty row
*   save the file

Pictures of this process are *how_it_looks_top.png*, *how_it_should_look_top.png*, *how_it_looks_player_row.png*, *how_it_should_look_player_row.png* and can be seen in the **BA Daten Original** folder in Google Drive


After checking and updating all the produced files for correctness, put them into the folder with the files name as its name, e.g. put the *1_def.csv* file in the **1_def** folder

In [ ]:
# add the team names and game IDs

for name in ["def", "misc", "passing", "passtypes", "poss", "summary", "shots"]:
  team_names_and_gameID(source + "/1_{}/1_{}.csv".format(name, name), name, teamnames_order)

In [ ]:
# read in all the data frames
summary_df = pd.read_csv(source + "/summary_final.csv")
passing_df = pd.read_csv(source + "/passing_final.csv")
passtypes_df = pd.read_csv(source + "/passtypes_final.csv")
def_df = pd.read_csv(source + "/def_final.csv")
poss_df = pd.read_csv(source + "/poss_final.csv")
misc_df = pd.read_csv(source + "/misc_final.csv")

# throw out irrelevant columns
new_passing_df = passing_df.iloc[:, 8:-2]
new_passtypes_df = passtypes_df.iloc[:, 8:-2]
new_def_df = def_df.iloc[:, 8:-2]
new_poss_df = poss_df.iloc[:, 8:-2]
new_misc_df = misc_df.iloc[:, 8:-2]

In [ ]:
# produce the final file for the specific league and season and save it as a csv file
final = pd.concat([summary_df, new_passing_df, new_passtypes_df, new_def_df, new_poss_df, new_misc_df], axis=1, join='inner')
final = final.iloc[:, 1:]
# change the name of the final file according to the league and season it belongs to
final.to_csv(source + "/final_BL_17_18.csv")

In [ ]:
# save the shots data frame as a csv file
shots = pd.read_csv(source  + "/shots_final.csv")
shots.iloc[:, 1:]

In [ ]:
#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

pd.reset_option("display.max_columns")
pd.reset_option("display.max_rows")

Repeat the previous steps for each league and season until it has been done for all seasons and leagues, only then continue

# Werte

## Spalten

In [ ]:
# old and new names of the headers

col_names_original_BL_18_19 = ["Unnamed:0", "ID", "Player","#","Nation","Pos","Age","Min","Gls","Ast","PK","PKatt","Sh","SoT","CrdY","CrdR","Touches","Press","Tkl","Int","Blocks","xG","npxG","xA","SCA","GCA","Cmp","Att","Cmp%","Prog","Carries","Prog","Succ","Att","Team","GameID",
                               "Cmp","Att","Cmp%","TotDist","PrgDist","Cmp","Att","Cmp%","Cmp","Att","Cmp%","Cmp","Att","Cmp%","Ast","xA","KP","1/3","PPA","CrsPA","Prog",
                               "Att","Live","Dead","FK","TB","Press","Sw","Crs","CK","In","Out","Str","Ground","Low","High","Left","Right","Head","TI","Other","Cmp","Off","Out","Int","Blocks",
                               "Tkl","TklW","Def 3rd","Mid 3rd","Att 3rd","Tkl","Att","Tkl%","Past","Press","Succ","%","Def 3rd","Mid 3rd","Att 3rd","Blocks","Sh","ShSv","Pass","Int","Tkl+Int","Clr","Err",
                               "Touches","Def Pen","Def 3rd","Mid 3rd","Att 3rd","Att Pen","Live","Succ","Att","Succ%","#Pl","Megs","Carries","TotDist","PrgDist","Prog","1/3","CPA","Mis","Dis","Targ","Rec","Rec%",
                               "CrdY","CrdR","2CrdY","Fls","Fld","Off","Crs","Int","TklW","PKwon","PKcon","OG","Recov","Won","Lost","Won%",'League']

col_names_new_names_BL_18_19 = ["Index", "PlayerID", "Player","#","Nation","Pos","Age","Min","Gls","Ast","PK","PKatt","Sh","SoT","CrdY","CrdR","Touches","Press","Tkl","Int","Blocks","xG","npxG","xA","SCA","GCA","PassesCmp","PassesAtt","PassesCmp%","PassesProg","Carries","CarriesProg","DribblesSucc","DribblesAtt","Squad","GameID",
                                "PassesCmp2","PassesAtt2","PassesCmp%2","PassesTotDist","PassesPrgDist","PassesShortCmp","PassesShortAtt","PassesShortCmp%","PassesMediumCmp","PassesMediumAtt","PassesMediumCmp%","PassesLongCmp","PassesLongAtt","PassesLongCmp%","Ast2","xA2","KP","Passes1/3","PPA","CrsPA","PassesProg2",
                                "PassesAtt3","PassesLive","PassesDead","PassesFK","PassesTB","PassesPress","PassesSw","PassesCrs","PassesCK","CrsIn","CrsOut","CrsStr","PassesGround","PassesLow","PassesHigh","PassesLeft","PassesRight","PassesHead","PassesTI","PassesOther","PassesCmp3","PassesOff","PassesOut","PassesInt","PassesBlocked",
                                "Tkl2","TklW","Tkl Def 3rd","Tkl Mid 3rd","Tkl Att 3rd","TklvsDribbles","AttvsDribbles","Tkl%vsDribbles","PastvsDribbles","Press2","PressSucc","Press%","Press Def 3rd","Press Mid 3rd","Press Att 3rd","Blocks2","BlocksSh","BlocksShSv","BlocksPass","Int2","Tkl+Int","Clr","Err",
                                "Touches2","Touches Def Pen","Touches Def 3rd","Touches Mid 3rd","Touches Att 3rd","Touches Att Pen","TouchesLive","DribblesSucc2","DribblesAtt2","DribblesSucc%","Dribbles#Pl","DribblesMegs","Carries2","CarriesTotDist","CarriesPrgDist","CarriesProg2","Carries1/3","CPA","CarriesMiscon","CarriesDispos","RecTarg","Rec","Rec%",
                                "CrdY2","CrdR2","2CrdY","Fls","Fld","Off","Crs2","Int2","TklW2","PKwon","PKcon","OG","Recov","AerialWon","AerialLost","AerialWon%",'League']

col_names_new_order_BL_18_19 = ['League',"GameID","PlayerID","Squad","Player","#","Nation","Pos","Age","Min","Gls","PK","PKatt","Sh","SoT","xG","npxG","SCA","GCA",
                                "PassesCmp","PassesAtt","PassesCmp%","PassesTotDist","PassesPrgDist","PassesShortCmp","PassesShortAtt","PassesShortCmp%","PassesMediumCmp","PassesMediumAtt","PassesMediumCmp%","PassesLongCmp","PassesLongAtt","PassesLongCmp%","Ast","xA","KP","Passes1/3","PPA","CrsPA","PassesProg",
                                "PassesLive","PassesDead","PassesFK","PassesTB","PassesPress","PassesSw","PassesCrs","PassesCK","CrsIn","CrsOut","CrsStr","PassesGround","PassesLow","PassesHigh","PassesLeft","PassesRight","PassesHead","PassesTI","PassesOther","PassesCmp3","PassesOff","PassesOut","PassesInt","PassesBlocked",
                                "Tkl","TklW","Tkl Def 3rd","Tkl Mid 3rd","Tkl Att 3rd","TklvsDribbles","AttvsDribbles","Tkl%vsDribbles","PastvsDribbles","Press","PressSucc","Press%","Press Def 3rd","Press Mid 3rd","Press Att 3rd","Blocks","BlocksSh","BlocksShSv","BlocksPass","Int","Tkl+Int","Clr","Err",
                                "Touches","Touches Def Pen","Touches Def 3rd","Touches Mid 3rd","Touches Att 3rd","Touches Att Pen","TouchesLive","DribblesSucc","DribblesAtt","DribblesSucc%","Dribbles#Pl","DribblesMegs","Carries","CarriesTotDist","CarriesPrgDist","CarriesProg","RecTarg","Rec","Rec%","CarriesMiscon","CarriesDispos",
                                "CrdY","CrdR","2CrdY","Fls","Fld","Off","PKwon","PKcon","OG","Recov","AerialWon","AerialLost","AerialWon%"]

col_names_original_others = ["Unnamed:0", "ID","Player","#","Nation","Pos","Age","Min","Gls","Ast","PK","PKatt","Sh","SoT","CrdY","CrdR","Touches","Press","Tkl","Int","Blocks","xG","npxG","xA","SCA","GCA","Cmp","Att","Cmp%","PrgDist","Carries","PrgDist.1","Succ","Att.1","Team","GameID",
                             "Cmp","Att","Cmp%","TotDist","PrgDist","Cmp.1","Att.1","Cmp%.1","Cmp.2","Att.2","Cmp%.2","Cmp.3","Att.3","Cmp%.3","Ast","xA","KP","1/3","PPA","CrsPA","Prog",
                             "Att","Live","Dead","FK","TB","Press","Sw","Crs","CK","In","Out","Str","Ground","Low","High","Left","Right","Head","TI","Other","Cmp","Off","Out.1","Int","Blocks",
                             "Tkl","TklW","Def 3rd","Mid 3rd","Att 3rd","Tkl.1","Att","Tkl%","Past","Press","Succ","%","Def 3rd.1","Mid 3rd.1","Att 3rd.1","Blocks","Sh","ShSv","Pass","Int","Tkl+Int","Clr","Err",
                             "Touches","Def Pen","Def 3rd","Mid 3rd","Att 3rd","Att Pen","Live","Succ","Att","Succ%","#Pl","Megs","Carries","TotDist","PrgDist","Targ","Rec","Rec%","Miscon","Dispos",
                             "CrdY","CrdR","2CrdY","Fls","Fld","Off","Crs","Int","TklW","PKwon","PKcon","OG","Recov","Won","Lost","Won%",'League']

col_names_new_names_others = ["Index", "PlayerID", "Player","#","Nation","Pos","Age","Min","Gls","Ast","PK","PKatt","Sh","SoT","CrdY","CrdR","Touches","Press","Tkl","Int","Blocks","xG","npxG","xA","SCA","GCA","PassesCmp","PassesAtt","PassesCmp%","PassesProg","Carries","CarriesProg","DribblesSucc","DribblesAtt","Squad","GameID",
                              "PassesCmp2","PassesAtt2","PassesCmp%2","PassesTotDist","PassesPrgDist","PassesShortCmp","PassesShortAtt","PassesShortCmp%","PassesMediumCmp","PassesMediumAtt","PassesMediumCmp%","PassesLongCmp","PassesLongAtt","PassesLongCmp%","Ast2","xA2","KP","Passes1/3","PPA","CrsPA","PassesProg2",
                              "PassesAtt3","PassesLive","PassesDead","PassesFK","PassesTB","PassesPress","PassesSw","PassesCrs","PassesCK","CrsIn","CrsOut","CrsStr","PassesGround","PassesLow","PassesHigh","PassesLeft","PassesRight","PassesHead","PassesTI","PassesOther","PassesCmp3","PassesOff","PassesOut","PassesInt","PassesBlocked",
                              "Tkl2","TklW","Tkl Def 3rd","Tkl Mid 3rd","Tkl Att 3rd","TklvsDribbles","AttvsDribbles","Tkl%vsDribbles","PastvsDribbles","Press2","PressSucc","Press%","Press Def 3rd","Press Mid 3rd","Press Att 3rd","Blocks2","BlocksSh","BlocksShSv","BlocksPass","Int2","Tkl+Int","Clr","Err",
                              "Touches2","Touches Def Pen","Touches Def 3rd","Touches Mid 3rd","Touches Att 3rd","Touches Att Pen","TouchesLive","DribblesSucc2","DribblesAtt2","DribblesSucc%","Dribbles#Pl","DribblesMegs","Carries2","CarriesTotDist","CarriesPrgDist","RecTarg","Rec","Rec%","CarriesMiscon","CarriesDispos",
                              "CrdY2","CrdR2","2CrdY","Fls","Fld","Off","Crs2","Int2","TklW2","PKwon","PKcon","OG","Recov","AerialWon","AerialLost","AerialWon%",'League']

col_names_new_order_others = ['League',"GameID","PlayerID","Squad","Player","#","Nation","Pos","Age","Min","Gls","PK","PKatt","Sh","SoT","xG","npxG","SCA","GCA",
                              "PassesCmp","PassesAtt","PassesCmp%","PassesTotDist","PassesPrgDist","PassesShortCmp","PassesShortAtt","PassesShortCmp%","PassesMediumCmp","PassesMediumAtt","PassesMediumCmp%","PassesLongCmp","PassesLongAtt","PassesLongCmp%","Ast","xA","KP","Passes1/3","PPA","CrsPA","PassesProg",
                              "PassesLive","PassesDead","PassesFK","PassesTB","PassesPress","PassesSw","PassesCrs","PassesCK","CrsIn","CrsOut","CrsStr","PassesGround","PassesLow","PassesHigh","PassesLeft","PassesRight","PassesHead","PassesTI","PassesOther","PassesCmp3","PassesOff","PassesOut","PassesInt","PassesBlocked",
                              "Tkl","TklW","Tkl Def 3rd","Tkl Mid 3rd","Tkl Att 3rd","TklvsDribbles","AttvsDribbles","Tkl%vsDribbles","PastvsDribbles","Press","PressSucc","Press%","Press Def 3rd","Press Mid 3rd","Press Att 3rd","Blocks","BlocksSh","BlocksShSv","BlocksPass","Int","Tkl+Int","Clr","Err",
                              "Touches","Touches Def Pen","Touches Def 3rd","Touches Mid 3rd","Touches Att 3rd","Touches Att Pen","TouchesLive","DribblesSucc","DribblesAtt","DribblesSucc%","Dribbles#Pl","DribblesMegs","Carries","CarriesTotDist","CarriesPrgDist","CarriesProg","RecTarg","Rec","Rec%","CarriesMiscon","CarriesDispos",
                              "CrdY","CrdR","2CrdY","Fls","Fld","Off","PKwon","PKcon","OG","Recov","AerialWon","AerialLost","AerialWon%"]


shots_names_original = ['Unnamed: 0','ID','Minute','Player','Squad','Outcome','Distance','Body Part','Notes','Player.1','Event','Player.2','Event.1','GameID','Minute Copy','Leg','Minute as int','League']

shots_names_new_names = ["Index", "ShotID", "Minute", "Player", "Squad", "Outcome", "Distance", "Body Part", "Notes", "Player1", "Event1", "Player2", "Event2","GameID", "Minute Copy", "Leg",'intMinute','League']

shots_names_new_order = ['League',"GameID", "ShotID", "intMinute", "Leg", "Player", "Squad", "Outcome", "Distance", "Body Part", "Notes", "Player1", "Event1", "Player2","Event2"]


matches_names_original_BL_17_18 = ['Round','Wk','Day','Date','Time','Home','xG','Score','xG.1','Away','Attendance','Venue','Referee','Match Report','Notes',
                                   'Possession Home','Possession Away','Temperature','Rain','Humidity','ID','Home Score','Away Score','Home Team Rank','Away Team Rank','League']

matches_names_new_names_BL_17_18 = ['Round','Wk','Day','Date','Time','Home','Home xG','Score','Away xG','Away','Attendance','Venue','Referee','Match Report','Notes',
                                    'Possession Home','Possession Away','Temperature','Rain','Humidity','GameID','Home Score','Away Score','Home Team Rank','Away Team Rank','League']

matches_names_new_order_BL_17_18 = ['League','GameID','Wk','Day','Date','Time','Home','Home xG','Home Score','Away Score','Away xG','Away','Attendance','Venue','Referee','Match Report',
                                    'Possession Home','Possession Away','Temperature','Rain','Humidity','Home Team Rank','Away Team Rank']

matches_names_original_others = ['Wk','Day','Date','Time','Home','xG','Score','xG.1','Away','Attendance','Venue','Referee','Match Report','Notes',
                                   'Possession Home','Possession Away','Temperature','Rain','Humidity','ID','Home Score','Away Score','Home Team Rank','Away Team Rank','League']

matches_names_new_names_others = ['Wk','Day','Date','Time','Home','Home xG','Score','Away xG','Away','Attendance','Venue','Referee','Match Report','Notes',
                                  'Possession Home','Possession Away','Temperature','Rain','Humidity','GameID','Home Score','Away Score','Home Team Rank','Away Team Rank','League']

matches_names_new_order_others = ['League','GameID','Wk','Day','Date','Time','Home','Home xG','Home Score','Away Score','Away xG','Away','Attendance','Venue','Referee','Match Report',
                                  'Possession Home','Possession Away','Temperature','Rain','Humidity','Home Team Rank','Away Team Rank']


tables_names_original = ['Rk','Squad','MP','W','D','L','GF','GA','GDiff','Pts','xG','xGA','xGDiff','xGDiff/90','Attendance','Top Team Scorer','Goalkeeper','Notes','League']

tables_names_new_names = ['Rk','Squad','MP','W','D','L','GF','GA','GDiff','Pts','xG','xGA','xGDiff','xGDiff/90','Attendance','Top Team Scorer','Goalkeeper','Notes','League']

tables_names_new_order = ['League','Rk','Squad','MP','W','D','L','GF','GA','GDiff','Pts','xG','xGA','xGDiff','xGDiff/90','Attendance','Top Team Scorer','Goalkeeper','Notes']

## Wetterdaten

In [ ]:
# create an array with possession, temperature, amount of rain and humidity values

# BL 17/18

possession_home_team_BL_17_18 = [51,63,62,30,36,53,38,56,53,70,54,50,29,59,53,66,67,48,33,68,38,34,49,18,30,53,39,
                                 58,76,52,45,65,52,59,64,73,55,35,61,33,55,47,37,50,44,57,64,53,43,63,44,79,65,53,
                                 42,41,55,43,52,55,43,32,49,45,69,52,46,38,54,65,53,46,59,58,48,57,40,26,47,57,53,
                                 53,47,47,55,35,69,71,61,71,49,52,61,63,73,40,49,38,59,39,48,51,67,72,49,58,50,52,
                                 48,45,57,51,53,47,30,42,55,59,28,36,43,68,59,65,54,43,60,55,61,66,52,52,35,38,59,
                                 48,44,41,65,57,54,48,43,81,54,37,44,63,37,38,46,51,73,45,62,48,46,52,46,51,41,52,
                                 38,52,41,44,47,40,52,64,45,48,57,62,52,71,59,60,69,57,49,46,54,37,37,45,46,38,56,
                                 50,64,62,40,57,56,57,32,39,68,52,36,25,51,35,60,55,44,42,52,39,59,74,65,54,55,65,
                                 50,66,50,54,47,48,50,59,39,54,46,60,55,75,53,45,46,42,56,56,50,53,62,58,60,50,38,
                                 43,56,65,44,43,60,53,44,43,44,55,29,60,51,61,50,64,47,60,52,41,53,54,69,42,43,48,
                                 59,54,54,43,26,57,43,43,58,54,57,28,58,49,42,64,39,35,64,36,40,48,37,66,51,35,44,
                                 45,54,45,41,47,79,47,48,47]


possession_away_team_BL_17_18 = [100-i for i in possession_home_team_BL_17_18]


temperature_BL_17_18 = [16,22,21,20,18,22,17,19,17,19,28,29,24,26,28,23,23,17,14,17,18,11,18,13,14,20,16,
                        10,11,15,14,17,12,16,17,13,12,12,9,7,14,12,11,7,11,9,18,18,20,18,19,15,15,16,
                        19,20,13,19,19,15,17,15,14,11,18,19,20,17,20,17,22,15,14,15,17,16,15,11,11,8,9,
                        9,10,12,10,10,11,8,10,5,9,15,12,15,10,12,13,9,7,1,6,6,5,4,5,5,8,8,
                        4,5,5,6,5,6,4,5,3,-3,1,1,0,-2,1,-1,1,1,-1,3,2,2,2,1,-1,-1,2,
                        2,-2,1,-1,4,5,7,2,0,1,3,1,1,4,2,2,1,0,4,5,1,2,2,3,-1,4,2,
                        0,2,3,4,4,3,0,0,2,6,7,6,8,7,5,4,10,9,1,2,2,4,2,5,2,2,1,
                        -2,2,3,1,4,3,0,3,1,0,5,2,4,4,3,0,3,1,-2,1,1,4,-1,-3,0,-5,-4,
                        -2,-2,1,3,3,-2,-5,8,5,6,12,15,8,10,11,13,12,8,4,0,4,2,0,-3,0,2,-4,
                        5,13,13,10,12,5,3,6,7,6,22,18,21,22,12,25,19,12,12,18,20,20,16,13,20,12,14,
                        19,27,21,24,20,17,20,24,22,14,21,16,21,18,17,15,24,16,20,22,19,18,23,21,22,23,18,
                        25,25,25,21,23,22,22,18,25]


rain_BL_17_18 = [14.7,0,0.2,0,0,0,0,0,0,1,0,0,0,0.1,0,0,0,0,0,0,0.1,3.2,0.1,0,0,0.1,0,
                 0,0,1.1,1.5,0,0,0,0,0,0,0,0,0.4,0,0,0,0,0,0,0,0,0,0,0.3,0,0,0,
                 0.4,0.1,3.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0.3,0.2,0,0,0,0,
                 0,0.7,0,0,0.2,0,0,0.1,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,2,0,0,0,
                 0,0.2,1.2,0.5,0,0.1,0,0,0,0.1,0,0,0,0,0,0,0.1,0,0,0,0,0.3,0.1,0,0,1.8,0,
                 0,0,0,0,0,3.1,8.1,1.8,0.7,0,0.1,0.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0.1,0,0.3,2.7,1.7,3.2,0,0.2,0,0.1,0,0,0,0,0,0,0,0,0,0,0.1,0.1,0,0,0,0.1,0,
                 0,0,0,0.1,0,0.1,0,0.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0.4,0,0,0,0,0,0,0,0,0.1,0.7,0,0.1,0.4,0,0.1,0,0.9,0,0,0.2,0,0,0,0,0,0,
                 0.4,0,0,0,0,0,0.2,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0.3,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.2,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0.9,0.2,0.1]


humidity_BL_17_18 = [95,43,49,57,66,43,55,49,64,73,48,46,55,54,56,55,48,72,93,63,54,96,65,85,78,53,58,
                     87,77,74,66,42,66,49,44,57,74,74,91,95,70,87,80,89,75,90,57,66,52,58,55,72,86,65,
                     77,66,85,69,74,89,54,78,67,85,71,74,57,79,56,75,54,91,60,74,75,66,83,96,73,80,91,
                     80,76,69,67,78,67,62,65,79,80,73,68,68,81,72,73,67,88,93,64,69,70,82,82,81,67,85,
                     94,83,88,71,85,66,78,80,83,92,76,69,92,73,74,82,85,99,93,78,78,82,64,78,95,90,89,
                     92,94,82,92,75,89,82,78,77,93,80,90,91,75,89,82,91,90,95,68,79,83,79,78,77,71,73,
                     93,84,78,83,84,90,95,91,84,88,76,81,69,71,85,83,80,83,90,74,75,60,67,65,88,72,91,
                     86,72,93,85,65,91,88,84,91,88,56,93,75,78,66,90,70,63,40,41,46,35,61,56,22,47,62,
                     61,52,70,86,60,41,44,75,87,92,76,73,92,72,82,71,87,91,83,35,83,49,53,39,42,48,60,
                     83,44,42,52,43,83,94,62,70,62,44,52,37,39,70,26,54,73,95,47,42,49,65,72,38,91,67,
                     62,34,61,42,48,50,55,46,56,67,38,47,55,54,68,61,43,80,62,57,52,55,40,41,41,33,47,
                     29,31,35,66,52,58,61,72,46]


# BL 17/18

possession_home_team_BL_18_19 = [67,58,50,55,52,51,49,48,47,56,50,40,44,57,57,33,68,56,60,75,33,53,34,60,51,56,44,
                                 54,49,71,50,55,40,31,51,47,54,63,58,43,46,44,47,55,72,32,56,39,44,39,55,43,42,51,
                                 49,65,54,48,50,71,44,69,59,54,31,51,54,65,46,45,61,54,41,61,57,34,42,73,44,68,59,
                                 53,71,57,59,44,57,49,57,37,32,51,58,53,50,49,42,43,52,58,46,59,35,77,57,55,47,62,
                                 52,45,59,36,67,49,46,55,31,69,42,46,76,52,61,41,70,71,46,48,47,37,61,28,52,42,48,
                                 53,57,27,45,54,45,60,51,49,54,57,45,56,62,61,39,43,65,44,61,46,64,49,49,47,43,45,
                                 47,44,62,61,45,53,49,71,53,47,52,63,41,40,49,38,52,69,29,69,51,52,51,59,69,41,43,
                                 29,45,51,49,61,38,42,83,28,68,61,41,41,58,74,51,33,52,29,72,45,60,50,54,38,51,59,
                                 56,67,75,36,60,46,63,39,45,55,40,61,50,60,47,70,63,69,36,53,30,31,56,50,50,59,50,
                                 31,61,63,56,38,59,59,42,53,52,42,52,32,60,61,59,33,45,69,63,55,52,61,40,42,60,59,
                                 27,69,47,50,45,40,34,46,33,57,53,70,43,52,49,51,39,81,40,74,59,39,49,60,65,36,57,
                                 40,50,53,31,58,61,63,36,56]


possession_away_team_BL_18_19 = [100-i for i in possession_home_team_BL_18_19]


temperature_BL_18_19 = [14,18,19,17,14,18,14,21,19,13,21,11,22,21,18,15,17,21,16,19,20,21,20,19,16,22,16,
                        11,15,16,17,17,18,12,11,12,10,4,8,6,15,11,12,13,14,10,17,15,16,15,14,13,18,12,
                        12,23,22,24,22,14,18,20,9,14,15,15,13,15,13,12,14,11,6,9,10,9,10,7,4,6,4,
                        5,19,10,10,8,10,9,12,9,9,10,11,12,13,13,13,13,13,4,2,5,7,5,3,4,6,4,
                        7,7,7,6,9,7,9,11,10,8,6,9,6,8,10,8,8,7,-3,-3,-2,1,0,-1,-1,-1,0,
                        5,1,5,1,8,2,1,5,5,10,7,6,10,8,8,9,6,11,0,-1,2,3,2,-2,0,0,1,
                        -2,4,5,6,4,5,5,3,5,1,1,7,5,4,4,1,-1,1,8,10,10,11,10,10,7,9,8,
                        4,14,12,15,17,11,13,12,7,4,4,9,11,12,10,12,13,9,6,12,8,6,10,12,11,12,8,
                        5,13,11,10,11,12,13,7,5,12,12,13,10,8,4,10,10,14,13,17,18,18,16,17,14,11,14,
                        9,20,17,16,17,20,13,12,19,4,7,7,6,7,6,9,12,9,20,25,23,19,22,18,23,20,11,
                        5,12,17,11,13,13,8,11,10,11,10,9,12,11,7,10,9,10,12,13,8,13,15,9,15,15,14,
                        25,21,25,23,24,19,17,23,20]


rain_BL_18_19 = [0.3,0,1.5,0,0,0,0,0,0,0,0,0.9,0,0,0,0,0.1,0,0,0,0.1,0,0,0.1,0,0,0,
                 0,0,0.1,0,0,0,0,3.7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0.1,0,0.3,1.2,0,0,0,0,0.7,0,0.1,0,0.2,0,0,0,
                 0.1,0,0,0.1,0,0,0,0,7.5,0,0.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0.5,0,0,0.4,0.8,0,0.4,1.6,0,0,2.4,0,0,0,0,0,0,0,0,0,0,
                 0,0,0.2,0.2,2.3,0.1,0.8,0.3,0.4,0,0.2,0,0,0,0.2,0.2,3.3,0.9,0.1,1.1,0.1,0.2,0.1,1.8,0,1,1.2,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.5,0,0.1,0,0.1,0,0.2,0,0.9,
                 0,0.3,1.3,2.4,0.2,0.1,0.2,3.1,0.3,6.3,0.1,0,0.1,2.5,3.9,1.9,0.6,0.3,0,0,0,0,0.3,0,0,0,0.3,
                 0,0,0,0,0,0,0,0.1,0.1,0,0.8,0.4,0.9,0.9,0.1,0,0,0,0,0,0,0,0,0,0,0,0,
                 1.9,2,0.3,0.6,1.7,1.7,1.9,0.7,0.3,0,1.1,1.3,0.3,0,0.2,0.1,0.1,0.2,4.6,1.1,3.8,1.5,0.2,2.6,0.5,0.3,0,
                 0,0,0,0,0.1,0.5,1.1,0,0.1]


humidity_BL_18_19 = [89,45,53,54,77,47,68,34,40,73,31,94,33,42,54,64,83,47,53,64,47,42,48,54,64,46,75,
                     76,57,49,46,42,37,75,81,72,77,73,74,69,41,58,44,50,40,64,37,47,32,41,45,41,34,86,
                     90,44,50,45,60,80,73,61,76,77,58,63,76,51,66,67,58,65,84,69,55,54,56,56,80,79,76,
                     92,85,49,48,62,84,71,77,87,89,71,67,87,69,81,81,71,75,74,91,92,82,91,90,85,78,93,
                     85,86,81,85,71,80,86,93,92,79,82,66,67,76,67,73,74,79,73,69,67,59,60,68,60,66,85,
                     73,90,68,95,82,91,89,89,95,74,88,86,74,85,79,78,91,86,75,71,46,49,58,78,61,57,52,
                     67,87,93,92,94,86,95,89,77,87,95,78,85,89,90,93,96,87,77,64,72,61,61,63,73,86,76,
                     88,46,52,48,44,63,45,62,71,85,32,41,38,41,39,29,33,67,89,64,74,58,67,61,78,58,80,
                     74,49,61,82,63,58,61,83,66,87,62,62,75,85,88,61,62,65,70,43,45,43,56,41,60,50,59,
                     82,50,50,55,50,34,65,73,46,59,54,62,76,62,51,52,37,57,42,28,28,52,37,42,40,36,59,
                     91,56,54,70,51,75,79,59,56,57,55,74,40,39,61,48,54,48,86,75,94,71,64,88,65,80,42,
                     34,59,35,42,38,57,78,40,60]


# PD 17/18

possession_home_team_PD_17_18 = [53,48,54,53,63,63,60,32,43,53,60,47,27,46,57,61,58,56,45,66,60,71,49,59,75,42,49,67,48,50, 
                                57,47,32,60,64,48,45,56,49,37,49,60,56,58,57,60,58,54,64,32,40,33,46,40,39,41,70,49,58,67,
                                64,61,63,63,41,56,49,41,59,63,57,44,32,27,32,55,52,54,58,60,50,68,52,64,52,66,40,58,62,60,
                                46,46,40,61,44,38,68,36,62,40,73,58,50,37,52,50,49,51,56,60,55,44,33,51,35,47,46,61,47,45,
                                58,42,59,52,45,50,53,45,30,52,62,65,55,57,36,40,35,56,56,55,37,48,59,60,39,67,72,61,32,53,
                                60,40,52,63,46,65,61,61,46,57,35,49,51,42,53,45,45,54,54,61,59,55,48,32,71,57,61,51,58,
                                43,43,58,64,53,38,39,45,50,74,51,55,55,58,59,48,63,55,48,68,43,42,42,45,37,43,46,65,77,42,
                                63,44,54,28,29,50,29,40,50,62,55,69,47,46,53,55,77,70,59,39,50,56,47,48,33,68,59,50,56,33,
                                39, # Rest of matchday 16 
                                51,51,63,45,63,54,59,35,60,32,40,45,43,49,48,65,50,51,37,64,64,49,31,60,75,31,57,60,40,58,
                                49,46,63,44,25,48,47,54,44,49,35,46,61,66,68,36,59,52,62,64,48,45,50,40,56,49,52,42,53,33,
                                48,49,48,48,69,42,68,51,46,49,43,58,63,48,42,60,65,57,39,40,37,48,56,63,52,65,52,57,52,58,
                                54,63,60,58,35,65,34,54,43,55,52,58,39,40,40,59,38,61,41,57,44,52,57,54,43,68,50,44,
                                56,35, # Rest of matchday 34
                                60,38,51,50,49,56,48,57,66,36,62,31,62,62,40,40,57,55,52,63]


possession_away_team_PD_17_18 = [100-i for i in possession_home_team_PD_17_18]


temperature_PD_17_18 = [30,25,24,21,29,20,24,19,24,23,20,29,25,21,25,23,27,23,23,23,25,22,26,27,18,18,19,17,23,24,
                        12,21,21,24,16,18,22,22,14,19,20,20,18,22,16,21,23,22,14,21,24,27,22,17,25,29,24,19,18,25,
                        16,19,32,21,23,17,20,21,23,22,21,23,27,18,20,23,23,21,24,24,25,27,21,21,22,17,15,14,14,14,
                        15,19,13,21,20,21,14,18,21,17,17,22,15,11,18,21,14,12,17,10,10,17,18,17,11,17,17,19,9,12,
                        13,8,17,18,15,12,9,13,10,11,12,9,8,9,7,7,8,6,18,5,5,13,15,14,12,13,16,15,16,11,
                        14,8,7,3,11,11,16,9,10,11,8,6,7,3,12,12,11,15,11,4,13,16,7,3,11,10,7,8,7,
                        5,11,5,8,8,13,5,10,8,7,7,12,15,16,15,11,16,11,12,11,8,10,15,10,11,12,14,11,12,9,
                        6,7,14,4,10,3,9,5,2,14,7,11,13,6,3,14,13,9,12,6,10,17,9,4,12,8,13,12,12,9,
                        8, # Rest of matchday 16
                        7,12,10,6,9,14,9,11,11,9,4,-2,11,6,13,10,7,13,17,7,19,14,10,9,9,15,15,10,13,8,
                        13,14,18,10,12,17,11,17,11,6,11,10,16,7,15,11,13,9,15,7,15,11,17,11,15,17,14,14,11,12,
                        9,10,11,14,14,18,14,11,14,12,11,19,14,17,15,11,13,18,14,14,16,11,17,19,20,17,17,19,14,16,
                        14,20,19,23,20,18,16,13,17,20,12,13,16,11,11,18,9,11,16,19,15,18,17,17,22,18,18,18,
                        18,17, # Rest of matchday 34
                        11,7,11,16,10,17,20,12,18,15,21,24,18,24,17,18,23,17,21,19]


rain_PD_17_18 = [0,0,0,0,0,0,0,0,0,0,1.6,0,0,0,0,0,0,0,2.3,2,0,0,0,0,4.6,0,0.1,0,0,0,
                 1.2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0.1,2.7,0,1.1,0,2.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 9.5,0,0,0,0,0,0.1,0,0,0,0,0,0.1,0,0.3,0,0,0,0,0,0.2,0,0,0,0,0,0,0,0,0,
                 0,1.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,1.4,0,0,0,0,
                 0,0,1,0,1.4,0,0.1,0,2.6,0,0,0,0,0,0,0,0,0,0,0.3,1.6,0,0,0,0.1,0,0,0,0,0,
                 2.5,0,0,0,0,0,3,0,0,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.6,0,0,0,0,0,0,0,
                 0, # Rest of matchday 16
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.8,1,0,0,1.6,0,2.1,0,0,0.7,0,0.1,0,0,0.9,
                 0,0,0.2,0,0,0,0.1,0,0.2,0.1,0,0,0.3,0,5.1,0,0,0,0,0,0,0.7,0,0,0,0,0,0,0,0.1,
                 0,0,1.3,0,0,0,0.2,0,0,0,3,0,2.8,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0.1,0,0.3,0,2.6,0,0.8,0,0,0,0.6,0,0.2,0,0,0,0,0,0,0,0,0,0,0.7,
                 0,0, # Rest of matchday 34
                 0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]           


humidity_PD_17_18 = [22,81,50,71,40,66,71,75,72,81,88,33,50,92,75,77,64,72,53,53,13,40,41,30,75,72,60,78,48,50,
                     82,42,20,25,26,39,39,77,84,70,83,63,62,26,82,26,39,71,81,77,38,28,75,91,58,22,74,78,85,27,
                     93,59,28,72,40,90,66,68,62,45,80,58,21,71,35,61,63,73,30,66,40,47,63,70,43,56,36,38,67,75,
                     70,30,82,34,19,32,84,86,75,65,83,39,83,95,67,30,66,77,37,52,63,31,27,45,38,62,33,73,68,57,
                     95,85,45,49,56,57,71,52,57,48,49,27,39,38,76,32,31,83,74,59,97,65,58,88,73,82,70,57,56,56,
                     91,75,77,65,33,54,69,43,66,74,49,78,81,95,60,64,53,70,85,85,85,68,60,86,77,42,90,88,81,
                     64,58,66,73,81,39,84,63,88,71,68,50,31,43,87,80,50,92,81,95,78,83,33,55,51,49,48,56,63,79,
                     77,79,31,97,41,76,75,92,84,69,73,34,52,57,67,47,54,89,61,74,92,74,77,92,80,84,31,58,69,66,
                     43, # Rest of matchday 16
                     73,42,23,32,55,42,63,62,78,82,65,51,83,84,91,86,84,72,82,74,32,83,82,80,87,59,61,70,54,89,
                     81,80,60,75,86,48,55,69,73,87,52,73,47,76,91,44,40,83,36,52,34,62,74,70,35,34,67,63,54,74,
                     84,83,84,71,68,42,44,80,53,43,95,50,74,70,50,82,85,52,70,85,69,90,60,52,49,47,66,68,84,61,
                     64,57,60,43,63,73,64,94,83,58,80,41,81,52,74,31,86,71,68,63,74,64,64,71,36,65,66,58,
                     72,75, # Rest of matchday 34
                     76,73,70,32,64,82,41,38,37,55,53,32,71,43,75,63,43,71,39,67]
      

# PD 18/19

possession_home_team_PD_18_19 = [68,76,49,52,79,69,48,78,51,66,38,66,34,47,37,50,52,33,55,60,55,57,66,50,77,46,39,67,62,
                                 56,58,32,40,40,48,57,58,70,33,49,52,56,47,42,64,49,55,69,67,46,55,58,24,66,37,40,64,50,36,
                                 52,55,69,63,65,52,61,48,82,52,68,53,54,31,36,43,37,52,55,25,61,69,59,52,53,64,63,53,63,54,
                                 35, # Rest of matchday 3
                                 56,45,52,40,43,61,32,53,47,61,46,58,54,32,71,59,52,52,58,42,46,50,39,50,58,51,56,58,51,52,
                                 50,39,49,49,32,64,49,60,36,48,36,58,68,41,59,57,44,62,46,47,65,49,55,53,45,59,45,68,60,51,
                                 80,29,56,69,57,49,35,55,43,47,66,66,58,38,52,42,49,47,40,46,56,53,59,46,35,54,57,59,24,65,
                                 39,49,63,55,57,58,45,58,72,41,45,68,51,55,59,50,49,37,72,56,55,65,54,45,46,36,40,67,38,51,
                                 43,56,47,68,45,42,54,57,70,62,51,49,35,61,65,31,42,58,34,48,58,58,48,61,61,56,52,43,66,51,
                                 67,44,40,48,47,50,51,49,40,46,54,54,60,29,49,61,64,61,51,38,65,43,46,68,57,39,59,49,52,39,
                                 60,56,54,47,54,62,50,54,64,56,49,57,68,63,43,40,47,60,40,71,50,48,33,42,44,52,33,63,55,36,
                                 55,56,51,64,46,49,39,55,49,56,56,28,56,39,59,46,35,58,54,30,45,61,51,56,51,51,36,56,66,66,
                                 48,51,24,56,58,48,32,49,64,34,63,58,48,62,49,44,50,46,50,53,64,37,50,50,35,43,46,59,40,52,
                                 59,42,78,45,61,68,42,50,51,54,47,42,38,48,49,52,43,47,47,40]


possession_away_team_PD_18_19 = [100-i for i in possession_home_team_PD_18_19]


temperature_PD_18_19 = [21,28,25,24,25,21,27,28,25,18,27,27,16,26,18,24,29,19,24,20,29,18,25,26,29,31,29,25,26,
                        22,27,24,23,19,27,26,27,28,19,22,30,24,24,30,27,31,30,29,26,22,13,23,21,21,21,28,23,17,16,
                        22,19,25,16,23,24,25,21,25,16,17,23,28,11,21,14,20,27,18,18,16,20,22,20,20,19,19,18,20,13,
                        19, # Rest of matchday 3
                        13,11,10,8,14,8,9,12,2,11,14,16,13,10,15,17,13,10,16,15,13,13,12,12,13,17,19,14,17,19,
                        7,13,16,7,8,12,19,13,16,10,6,11,11,9,8,17,16,14,8,14,10,12,19,14,12,13,14,12,15,9,
                        10,10,11,8,13,14,14,8,13,4,10,11,16,11,14,12,18,12,7,11,13,11,7,10,6,6,18,16,10,19,
                        6,7,16,11,12,9,12,13,11,8,6,7,5,11,14,14,12,9,6,6,15,13,12,13,6,12,9,8,11,4,
                        5,13,7,12,7,11,13,6,6,6,7,11,14,16,10,10,20,16,7,3,13,15,16,16,17,12,15,13,14,8,
                        20,15,21,14,12,15,19,17,11,12,15,15,21,16,19,19,24,20,20,12,10,14,18,17,15,13,20,24,22,12,
                        9,18,25,21,17,10,17,20,25,19,11,14,16,15,9,12,18,21,16,15,18,16,16,9,6,5,18,17,12,9,
                        16,11,10,14,9,13,11,19,6,10,19,19,21,20,17,18,17,16,22,17,13,19,18,16,15,13,16,16,14,17,
                        11,6,8,8,5,15,17,18,12,9,15,22,22,16,21,16,15,18,20,26,21,20,19,12,17,17,12,23,14,13,
                        22,20,20,27,32,26,20,26,16,15,18,18,20,25,14,11,7,11,18,12]


rain_PD_18_19 = [1.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,2.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.8,0,0,0,0,0,0.3,0,
                 0, # Rest of matchday 3
                 0,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.8,0,0,0.5,0,0,0,0,0,0,0.2,
                 0,0,0,0,0,0,0.4,0,0,0.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 1.5,0,0,0,0.2,0,0.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0.2,0,0.6,0,2,0,0.2,0.2,0,1,0.3,0,0,0,0,0,1,1,1.7,0,1.5,
                 0.5,0,5.9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0,0,0,
                 0,0,0,0,0,0.5,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0.2,0.2,0.1,1.3,0,0,0,0,0,
                 0,1.4,2.2,0.1,0.1,0,0,0,11.4,2.7,0,0,0,0,0,0.1,0,0,0,0,0,0,0,0,0.2,1.8,0.1,0.1,0.9,0,
                 11.3,0.2,0.2,9.8,8.6,0,0,0,0.1,0.1,0.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0,0.8,0,0,2.2]


humidity_PD_18_19 = [69,45,46,62,60,68,24,26,55,83,20,20,71,21,53,68,31,86,78,86,25,85,73,49,28,47,31,72,69,
                     57,36,58,71,77,37,61,28,41,89,55,27,60,63,25,30,40,47,25,68,64,69,60,61,71,77,43,39,77,83,
                     30,68,53,79,27,22,54,71,57,61,80,56,18,89,32,42,25,29,62,49,81,55,75,79,64,56,72,67,72,77,
                     38, # Rest of matchday 3
                     65,84,72,76,77,53,38,64,93,45,34,26,68,49,50,35,56,34,73,70,47,77,85,90,78,61,65,75,83,92,
                     77,66,38,67,71,65,68,55,67,81,66,91,69,53,60,45,57,61,86,56,65,60,43,68,62,79,53,84,53,85,
                     93,68,78,87,79,79,74,75,59,75,72,82,50,65,63,83,44,43,62,76,74,49,32,44,44,79,32,25,35,68,
                     61,58,35,52,55,50,77,50,57,92,59,63,70,98,67,47,54,57,61,81,52,41,46,64,60,67,70,76,39,95,
                     74,32,71,37,73,27,43,79,49,46,76,61,45,56,82,71,36,63,78,96,64,65,24,44,49,35,55,66,68,55,
                     51,30,39,59,69,37,30,39,75,66,40,70,34,42,39,41,42,40,32,65,78,62,36,52,46,87,42,50,52,71,
                     89,27,22,52,35,81,55,40,43,58,76,43,56,59,59,86,36,51,42,48,40,76,74,61,78,54,52,37,32,57,
                     40,47,49,53,64,44,46,30,95,92,38,29,26,56,63,64,65,54,51,57,77,55,57,50,57,77,47,45,80,59,
                     85,75,79,86,89,67,56,52,58,64,67,33,35,66,43,57,55,54,47,48,72,35,44,71,66,38,49,28,45,42,
                     36,56,54,16,30,19,63,19,65,73,51,22,43,22,43,60,92,80,31,93]

# Bearbeitung der fertigen Datensätze

## einlesen

In [ ]:
# create data frames from the files

source_BL_17_18 = "/content/drive/MyDrive/BA/BA Daten Original/Saison 17.18/BL"
source_PD_17_18 = "/content/drive/MyDrive/BA/BA Daten Original/Saison 17.18/PD"
source_BL_18_19 = "/content/drive/MyDrive/BA/BA Daten Original/Saison 18.19/BL"
source_PD_18_19 = "/content/drive/MyDrive/BA/BA Daten Original/Saison 18.19/PD"

bl_17_18 = pd.read_csv(source_BL_17_18 + "/final_BL_17_18.csv")
shots_BL_17_18 = pd.read_csv(source_BL_17_18 + "/shots_final.csv")
matches_BL_17_18 = pd.read_csv(source_BL_17_18 + "/101_Spieltage_BL_17_18.csv")
table_BL_17_18 = pd.read_csv(source_BL_17_18 + "/101_table.csv")

pd_17_18 = pd.read_csv(source_PD_17_18 + "/final_PD_17_18.csv")
shots_PD_17_18 = pd.read_csv(source_PD_17_18 + "/shots_final.csv")
matches_PD_17_18 = pd.read_csv(source_PD_17_18 + "/101_Spieltage_PD_17_18.csv")
table_PD_17_18 = pd.read_csv(source_PD_17_18 + "/101_table.csv")

bl_18_19 = pd.read_csv(source_BL_18_19 + "/final_BL_18_19.csv")
shots_BL_18_19 = pd.read_csv(source_BL_18_19 + "/shots_final.csv")
matches_BL_18_19 = pd.read_csv(source_BL_18_19 + "/101_Spieltage_BL_18_19.csv")
table_BL_18_19 = pd.read_csv(source_BL_18_19 + "/101_table.csv")

pd_18_19 = pd.read_csv(source_PD_18_19 + "/final_PD_18_19.csv")
shots_PD_18_19 = pd.read_csv(source_PD_18_19 + "/shots_final.csv")
matches_PD_18_19 = pd.read_csv(source_PD_18_19 + "/101_Spieltage_PD_18_19.csv")
table_PD_18_19 = pd.read_csv(source_PD_18_19 + "/101_table.csv")

## Daten hinzufügen

In [ ]:
# feed in the weather values

matches_BL_17_18["Possession Home"] = possession_home_team_BL_17_18
matches_BL_17_18["Possession Away"] = possession_away_team_BL_17_18
matches_BL_17_18["Temperature"] = temperature_BL_17_18
matches_BL_17_18["Rain"] = rain_BL_17_18
matches_BL_17_18["Humidity"] = humidity_BL_17_18

matches_PD_17_18["Possession Home"] = possession_home_team_PD_17_18
matches_PD_17_18["Possession Away"] = possession_away_team_PD_17_18
matches_PD_17_18["Temperature"] = temperature_PD_17_18
matches_PD_17_18["Rain"] = rain_PD_17_18
matches_PD_17_18["Humidity"] = humidity_PD_17_18

matches_BL_18_19["Possession Home"] = possession_home_team_BL_18_19
matches_BL_18_19["Possession Away"] = possession_away_team_BL_18_19
matches_BL_18_19["Temperature"] = temperature_BL_18_19
matches_BL_18_19["Rain"] = rain_BL_18_19
matches_BL_18_19["Humidity"] = humidity_BL_18_19

matches_PD_18_19["Possession Home"] = possession_home_team_PD_18_19
matches_PD_18_19["Possession Away"] = possession_away_team_PD_18_19
matches_PD_18_19["Temperature"] = temperature_PD_18_19
matches_PD_18_19["Rain"] = rain_PD_18_19
matches_PD_18_19["Humidity"] = humidity_PD_18_19

In [ ]:
# add the game IDs to the matches

# Bundesliga IDs

bl_id = []
for id in range(306):
  bl_id.append(id+1)


# Primera División IDs

pd_id = []
for id in range(380):
  pd_id.append(id+1)

matches_BL_17_18["ID"] = bl_id
matches_PD_17_18["ID"] = pd_id
matches_BL_18_19["ID"] = bl_id
matches_PD_18_19["ID"] = pd_id

In [ ]:
# split the score to be able to work with it

matches_BL_17_18["Home Score"] = [int(score[:1]) for score in matches_BL_17_18["Score"]]
matches_PD_17_18["Home Score"] = [int(score[:1]) for score in matches_PD_17_18["Score"]]
matches_BL_18_19["Home Score"] = [int(score[:1]) for score in matches_BL_18_19["Score"]]
matches_PD_18_19["Home Score"] = [int(score[:1]) for score in matches_PD_18_19["Score"]]

matches_BL_17_18["Away Score"] = [int(score[-1:]) for score in matches_BL_17_18["Score"]]
matches_PD_17_18["Away Score"] = [int(score[-1:]) for score in matches_PD_17_18["Score"]]
matches_BL_18_19["Away Score"] = [int(score[-1:]) for score in matches_BL_18_19["Score"]]
matches_PD_18_19["Away Score"] = [int(score[-1:]) for score in matches_PD_18_19["Score"]]

In [ ]:
def rank_team(table):
  """
    Creates dict with rank for the teams.

            Parameters:
                    table (csv file) : Table of the league and season after the last matchday

            Returns:
                    table_dict (dict) : Dictionary with the team and its rank for the specific season as key/value pair
  """

  table_dict = {}
  for rank in table["Rk"]:
    table_dict[table["Squad"][rank-1]] = rank
  return table_dict

# create the ranks
rank_BL_17_18 = rank_team(table_BL_17_18)
rank_PD_17_18 = rank_team(table_PD_17_18)
rank_BL_18_19 = rank_team(table_BL_18_19)
rank_PD_18_19 = rank_team(table_PD_18_19)

# add the home and away team rank to every match in the seasons
matches_BL_17_18["Home Team Rank"] = [rank_BL_17_18[team] for team in matches_BL_17_18["Home"] if team in rank_BL_17_18]
matches_BL_17_18["Away Team Rank"] = [rank_BL_17_18[team] for team in matches_BL_17_18["Away"] if team in rank_BL_17_18]

matches_PD_17_18["Home Team Rank"] = [rank_PD_17_18[team] for team in matches_PD_17_18["Home"] if team in rank_PD_17_18]
matches_PD_17_18["Away Team Rank"] = [rank_PD_17_18[team] for team in matches_PD_17_18["Away"] if team in rank_PD_17_18]

matches_BL_18_19["Home Team Rank"] = [rank_BL_18_19[team] for team in matches_BL_18_19["Home"] if team in rank_BL_18_19]
matches_BL_18_19["Away Team Rank"] = [rank_BL_18_19[team] for team in matches_BL_18_19["Away"] if team in rank_BL_18_19]

matches_PD_18_19["Home Team Rank"] = [rank_PD_18_19[team] for team in matches_PD_18_19["Home"] if team in rank_PD_18_19]
matches_PD_18_19["Away Team Rank"] = [rank_PD_18_19[team] for team in matches_PD_18_19["Away"] if team in rank_PD_18_19]

In [ ]:
# create a column for the leg in which the shot was taken

shots_BL_17_18["Minute Copy"] = shots_BL_17_18["Minute"] 
shots_BL_17_18["Minute Copy"] = [minute[:2] for minute in shots_BL_17_18["Minute"]]
shots_BL_17_18["Leg"] = [1 if int(minute) <= 45 else 2 for minute in shots_BL_17_18["Minute Copy"]]

shots_PD_17_18["Minute Copy"] = shots_PD_17_18["Minute"] 
shots_PD_17_18["Minute Copy"] = [minute[:2] for minute in shots_PD_17_18["Minute"]]
shots_PD_17_18["Leg"] = [1 if int(minute) <= 45 else 2 for minute in shots_PD_17_18["Minute Copy"]]

shots_BL_18_19["Minute Copy"] = shots_BL_18_19["Minute"] 
shots_BL_18_19["Minute Copy"] = [minute[:2] for minute in shots_BL_18_19["Minute"]]
shots_BL_18_19["Leg"] = [1 if int(minute) <= 45 else 2 for minute in shots_BL_18_19["Minute Copy"]]

shots_PD_18_19["Minute Copy"] = shots_PD_18_19["Minute"] 
shots_PD_18_19["Minute Copy"] = [minute[:2] for minute in shots_PD_18_19["Minute"]]
shots_PD_18_19["Leg"] = [1 if int(minute) <= 45 else 2 for minute in shots_PD_18_19["Minute Copy"]]

In [ ]:
# create a column for the minute values as integers

shots_BL_17_18["Minute as int"] = [int(minute[:2]) + int(minute[-2:]) if len(minute) == 5 else int(minute[:2]) + int(minute[-1]) if len(minute) == 4 else int(minute) for minute in shots_BL_17_18["Minute"]]
shots_PD_17_18["Minute as int"] = [int(minute[:2]) + int(minute[-2:]) if len(minute) == 5 else int(minute[:2]) + int(minute[-1]) if len(minute) == 4 else int(minute) for minute in shots_PD_17_18["Minute"]]
shots_BL_18_19["Minute as int"] = [int(minute[:2]) + int(minute[-2:]) if len(minute) == 5 else int(minute[:2]) + int(minute[-1]) if len(minute) == 4 else int(minute) for minute in shots_BL_18_19["Minute"]]
shots_PD_18_19["Minute as int"] = [int(minute[:2]) + int(minute[-2:]) if len(minute) == 5 else int(minute[:2]) + int(minute[-1]) if len(minute) == 4 else int(minute) for minute in shots_PD_18_19["Minute"]]

In [ ]:
# add the right league and season to every row

bl_17_18["League"] = "BL 17-18"
matches_BL_17_18["League"] = "BL 17-18"
shots_BL_17_18["League"] = "BL 17-18"
table_BL_17_18["League"] = "BL 17-18"

pd_17_18["League"] = "PD 17-18"
matches_PD_17_18["League"] = "PD 17-18"
shots_PD_17_18["League"] = "PD 17-18"
table_PD_17_18["League"] = "PD 17-18"

bl_18_19["League"] = "BL 18-19"
matches_BL_18_19["League"] = "BL 18-19"
shots_BL_18_19["League"] = "BL 18-19"
table_BL_18_19["League"] = "BL 18-19"

pd_18_19["League"] = "PD 18-19"
matches_PD_18_19["League"] = "PD 18-19"
shots_PD_18_19["League"] = "PD 18-19"
table_PD_18_19["League"] = "PD 18-19"

In [ ]:
# change the names and the order of the columns and leave the duplicate ones out

bl_17_18.columns = col_names_new_names_others
bl_17_18 = bl_17_18[col_names_new_order_others]

pd_17_18.columns = col_names_new_names_others
pd_17_18 = pd_17_18[col_names_new_order_others]

bl_18_19.columns = col_names_new_names_BL_18_19
bl_18_19 = bl_18_19[col_names_new_order_BL_18_19]

pd_18_19.columns = col_names_new_names_others
pd_18_19 = pd_18_19[col_names_new_order_others]

In [ ]:
# change the names and the order of the shots columns and leave the irrelevant ones out

shots_BL_17_18.columns = shots_names_new_names
shots_BL_17_18 = shots_BL_17_18[shots_names_new_order]

shots_PD_17_18.columns = shots_names_new_names
shots_PD_17_18 = shots_PD_17_18[shots_names_new_order]

shots_BL_18_19.columns = shots_names_new_names
shots_BL_18_19 = shots_BL_18_19[shots_names_new_order]

shots_PD_18_19.columns = shots_names_new_names
shots_PD_18_19 = shots_PD_18_19[shots_names_new_order]

In [ ]:
# rename the Minute column

shots_BL_17_18.rename(columns={"intMinute":"Minute"}, inplace=True)
shots_PD_17_18.rename(columns={"intMinute":"Minute"}, inplace=True)
shots_BL_18_19.rename(columns={"intMinute":"Minute"}, inplace=True)
shots_PD_18_19.rename(columns={"intMinute":"Minute"}, inplace=True)

In [ ]:
# change the names and the order of the matches columns and leave the irrelevant ones out

matches_BL_17_18.columns = matches_names_new_names_BL_17_18
matches_BL_17_18 = matches_BL_17_18[matches_names_new_order_BL_17_18]

matches_PD_17_18.columns = matches_names_new_names_others
matches_PD_17_18 = matches_PD_17_18[matches_names_new_order_others]

matches_BL_18_19.columns = matches_names_new_names_others
matches_BL_18_19 = matches_BL_18_19[matches_names_new_order_others]

matches_PD_18_19.columns = matches_names_new_names_others
matches_PD_18_19 = matches_PD_18_19[matches_names_new_order_others]

In [ ]:
# change the names and the order of the table columns and leave the irrelevant ones out

table_BL_17_18.columns = tables_names_new_names
table_BL_17_18 = table_BL_17_18[tables_names_new_order]

table_PD_17_18.columns = tables_names_new_names
table_PD_17_18 = table_PD_17_18[tables_names_new_order]

table_BL_18_19.columns = tables_names_new_names
table_BL_18_19 = table_BL_18_19[tables_names_new_order]

table_PD_18_19.columns = tables_names_new_names
table_PD_18_19 = table_PD_18_19[tables_names_new_order]

In [ ]:
# create dictionaries for renaming the teams

teamname_BL_dict = {'BMG':"M'Gladbach", 'BVB':'Dortmund', 'DUE': "Düsseldorf", 'FCA': "Augsburg", 'FCB': "Bayern Munich", 'FCN': "Nürnberg", 'FRA': "Frankfurt", 'HAN': "Hannover 96", 'HER': "Hertha BSC", 'HOF': "Hoffenheim",
                 'LEV':"Leverkusen", 'MAI':"Mainz 05", 'RBL': "RB Leipzig", 'SCF': "Freiburg", 'SCH': "Schalke 04", 'VFB': "Stuttgart", 'WER': "Werder Bremen", 'WOB':"Wolfsburg", "DUE":"Düsseldorf",
                 "HSV":"Hamburger SV", "KOE":"Köln"}

teamname_PD_dict = {"ALA":'Alavés', "ATH":'Athletic Club', "ATL":'Atlético Madrid', "BAR":'Barcelona', "BET":'Betis', "CEL":'Celta Vigo', "EIB":'Eibar', "ESP":'Espanyol', "GET":'Getafe', "GIR":'Girona',
                   "LAC":'La Coruña', "PAL":'Las Palmas', "LEG":'Leganés', "UDL":'Levante', "MAL":'Málaga', "MAD":'Real Madrid', "SOC":'Real Sociedad', "SEV":'Sevilla', "VAL":'Valencia', "VIL":'Villarreal',
                   "HUE":"Huesca", "RVA":"Valladolid", "RAY":"Rayo Vallecano"}

teamname_PD_shots_dict = {"Alav�s":"Alavés", "La Coru�a":"La Coruña", "M�laga":"Málaga"}

# use the dictionaries to rename the teams

bl_17_18["Squad"] = [teamname_BL_dict[team] for team in bl_17_18["Squad"] if team in teamname_BL_dict]
pd_17_18["Squad"] = [teamname_PD_dict[team] for team in pd_17_18["Squad"] if team in teamname_PD_dict]
bl_18_19["Squad"] = [teamname_BL_dict[team] for team in bl_18_19["Squad"] if team in teamname_BL_dict]
pd_18_19["Squad"] = [teamname_PD_dict[team] for team in pd_18_19["Squad"] if team in teamname_PD_dict]

for i in range(len(shots_PD_17_18)):
  if shots_PD_17_18["Squad"][i] in teamname_PD_shots_dict:
    shots_PD_17_18["Squad"][i] = teamname_PD_shots_dict[shots_PD_17_18["Squad"][i]]

for i in range(len(shots_PD_18_19)):
  if shots_PD_18_19["Squad"][i] in teamname_PD_shots_dict:
    shots_PD_18_19["Squad"][i] = teamname_PD_shots_dict[shots_PD_18_19["Squad"][i]]

In [ ]:
def df_to_csv(df, src, name):
  """
    Saves all matches, shots and stats csv data frames as a csv file.

            Parameters:
                    df (dataframe) : Data frame to be saved
                    src (str)      : Destination path for the csv version of the data frame
                    name (str)     : Name of the data frame

            Returns:
                    None
  """

  df.to_csv(src + "/{}.csv".format(name), index=False)
  print("The DataFrame {} was saved as {}.csv in the folder {}".format(name, name, src))


df_to_csv(bl_17_18, source_BL_17_18, "BL_17_18")
df_to_csv(shots_BL_17_18, source_BL_17_18, "shots_BL_17_18")
df_to_csv(matches_BL_17_18, source_BL_17_18, "matches_BL_17_18")
df_to_csv(table_BL_17_18, source_BL_17_18, "table_BL_17_18")

df_to_csv(pd_17_18, source_PD_17_18, "PD_17_18")
df_to_csv(shots_PD_17_18, source_PD_17_18, "shots_PD_17_18")
df_to_csv(matches_PD_17_18, source_PD_17_18, "matches_PD_17_18")
df_to_csv(table_PD_17_18, source_PD_17_18, "table_PD_17_18")

df_to_csv(bl_18_19, source_BL_18_19, "BL_18_19")
df_to_csv(shots_BL_18_19, source_BL_18_19, "shots_BL_18_19")
df_to_csv(matches_BL_18_19, source_BL_18_19, "matches_BL_18_19")
df_to_csv(table_BL_18_19, source_BL_18_19, "table_BL_18_19")

df_to_csv(pd_18_19, source_PD_18_19, "PD_18_19")
df_to_csv(shots_PD_18_19, source_PD_18_19, "shots_PD_18_19")
df_to_csv(matches_PD_18_19, source_PD_18_19, "matches_PD_18_19")
df_to_csv(table_PD_18_19, source_PD_18_19, "table_PD_18_19")

In [ ]:
# combine the files to have one file for each type

source = "/content/drive/MyDrive/BA/BA Daten Original/"

filenames = [bl_17_18, bl_18_19, pd_17_18, pd_18_19]
filenames_matches = [matches_BL_17_18, matches_BL_18_19, matches_PD_17_18, matches_PD_18_19]
filenames_shots = [shots_BL_17_18, shots_BL_18_19, shots_PD_17_18, shots_PD_18_19]
filenames_table = [table_BL_17_18, table_BL_18_19, table_PD_17_18, table_PD_18_19]

stats = pd.concat(filenames)
matches = pd.concat(filenames_matches)
shots = pd.concat(filenames_shots)
tables = pd.concat(filenames_table)

In [ ]:
# save the data frames as csv files

stats.to_csv(source + "stats.csv", index=False)
matches.to_csv(source + "matches.csv", index=False)
shots.to_csv(source + "shots.csv", index=False)
tables.to_csv(source + "tables.csv", index=False)

In [ ]:
matches = pd.read_csv(source + "matches.csv")
stats = pd.read_csv(source + "stats.csv")

In [ ]:
# create columns with the points for the home and away teams for every match

matches["Home Team Points"] = [3 if (matches["Home Score"][i] > matches['Away Score'][i]) else 1 if (matches["Home Score"][i] == matches['Away Score'][i]) else 0 for i in range(len(matches))]
matches["Away Team Points"] = [3 if (matches["Home Score"][i] < matches['Away Score'][i]) else 1 if (matches["Home Score"][i] == matches['Away Score'][i]) else 0 for i in range(len(matches))]

In [ ]:
# correct the names that contain "�"

player_names_dict = {"Aar�n Mart�n\Aaron-Martin":"Aarón Martín\Aaron-Martin", "Adri�n Gonz�lez\Adrian-Gonzalez":"Adrián González\Adrian-Gonzalez", "Adri�n L�pez\Adrian-Lopez":"Adrián López\Adrian-Lopez",
                     "Alberto L�pez\Alberto-Lopez":"Alberto López\Alberto-Lopez", "Aleix Garc�a\Aleix-Garcia":"Aleix García\Aleix-Garcia", "Alessandro Sch�pf\Alessandro-Schopf":"Alessandro Schöpf\Alessandro-Schopf",
                     "Andr�s Guardado\Andres-Guardado":"Andrés Guardado\Andres-Guardado", "Andr�s Prieto\Andres-Prieto":"Andrés Prieto\Andres-Prieto", "Antonio Barrag�n\Antonio-Barragan":"Antonio Barragán\Antonio-Barragan",
                     "A�ssa Mandi\Aissa-Mandi":"Aïssa Mandi\Aissa-Mandi", "Be�at Etxebarria\Benat-Etxebarria":"Beñat Etxebarria\Benat-Etxebarria", "Borja Bast�n\Borja-Baston":"Borja Bastón\Borja-Baston",
                     "Borja Garc�a\Borja-Garcia":"Borja García\Borja-Garcia", "Bruno Gonz�lez\Bruno-Gonzalez":"Bruno González\Bruno-Gonzalez", "Carlos Fern�ndez\Carlos-Fernandez":"Carlos Fernández\Carlos-Fernandez",
                     "Carlos S�nchez\Carlos-Sanchez":"Carlos Sánchez\Carlos-Sanchez", "Cl�ment Lenglet\Clement-Lenglet":"Clément Lenglet\Clement-Lenglet", "Dami�n Su�rez\Damian-Suarez":"Damián Suárez\Damian-Suarez",
                     "Daniel Carri�o\Daniel-Carrico":"Daniel Carriço\Daniel-Carrico", "David L�pez\David-Lopez":"David López\David-Lopez", "Diego Gonz�lez\Diego-Gonzalez":"Diego González\Diego-Gonzalez",
                     "Diego L�pez\Diego-Lopez":"Diego López\Diego-Lopez", "Diego Rol�n\Diego-Rolan":"Diego Rolán\Diego-Rolan", "Djen�\Djene":"Djené\Djene",
                     "D�dac Vil�\Didac-Vila":"Dídac Vilà\Didac-Vila", "Emre �olak\Emre-Colak":"Emre Çolak\Emre-Colak", "Fabian Sch�r\Fabian-Schar":"Fabian Schär\Fabian-Schar",
                     "Fabi�n Ruiz Pe�a\Fabian-Ruiz-Pena":"Fabián Ruiz Peña\Fabian-Ruiz-Pena", "Fay�al Fajr\Faycal-Fajr":"Fayçal Fajr\Faycal-Fajr", "Ferr�n Torres\Ferran-Torres":"Ferrán Torres\Ferran-Torres",
                     "Florian H�bner\Florian-Hubner":"Florian Hübner\Florian-Hubner", "Franck Rib�ry\Franck-Ribery":"Franck Ribéry\Franck-Ribery", "Franco V�zquez\Franco-Vazquez":"Franco Vázquez\Franco-Vazquez",
                     "Ger�nimo Rulli\Geronimo-Rulli":"Gerónimo Rulli\Geronimo-Rulli", "Gon�alo Guedes\Goncalo-Guedes":"Gonçalo Guedes\Goncalo-Guedes", "H�ctor Moreno\Hector-Moreno":"Héctor Moreno\Hector-Moreno",
                     "Ibai G�mez\Ibai-Gomez":"Ibai Gómez\Ibai-Gomez", "I�aki Williams\Inaki-Williams":"Iñaki Williams\Inaki-Williams", "I�igo C�rdoba\Inigo-Cordoba":"Iñigo Córdoba\Inigo-Cordoba",
                     "I�igo Lekue\Inigo-Lekue":"Iñigo Lekue\Inigo-Lekue", "I�igo Mart�nez\Inigo-Martinez":"Iñigo Martínez\Inigo-Martinez", "Javi Garc�a\Javi-Garcia":"Javi García\Javi-Garcia",
                     "Javi L�pez\Javi-Lopez":"Javi López\Javi-Lopez", "Javi Mart�nez\Javi-Martinez":"Javi Martínez\Javi-Martinez", "Jes�s Navas\Jesus-Navas":"Jesús Navas\Jesus-Navas",
                     "Joaqu�n Correa\Joaquin-Correa":"Joaquín Correa\Joaquin-Correa", "Joaqu�n\Joaquin":"Joaquín\Joaquin", "Jon�s Ramalho\Jonas-Ramalho":"Jonás Ramalho\Jonas-Ramalho",
                     "Jos� Luis Gay�\Jose-Luis-Gaya":"José Luis Gayà\Jose-Luis-Gaya", "Jos� Manuel Jurado\Jose-Manuel-Jurado":"José Manuel Jurado\Jose-Manuel-Jurado", "J�r�me Boateng\Jerome-Boateng":"Jérôme Boateng\Jerome-Boateng",
                     "K�vin Rodrigues\Kevin-Rodrigues":"Kévin Rodrigues\Kevin-Rodrigues", "Loren Mor�n\Loren-Moron":"Loren Morón\Loren-Moron", "Lucas P�rez\Lucas-Perez":"Lucas Pérez\Lucas-Perez",
                     "Luis Hern�ndez\Luis-Hernandez":"Luis Hernández\Luis-Hernandez", "L�o Baptist�o\Leo-Baptistao":"Léo Baptistão\Leo-Baptistao", "Manu Garc�a\Manu-Garcia":"Manu García\Manu-Garcia",
                     "Micka�l Cuisance\Mickael-Cuisance":"Mickaël Cuisance\Mickael-Cuisance", "Miguel Lay�n\Miguel-Layun":"Miguel Layún\Miguel-Layun", "Miguel �ngel Moy�\Miguel-Angel-Moya":"Miguel Ángel Moyá\Miguel-Angel-Moya",
                     "Mikel San Jos�\Mikel-San-Jose":"Mikel San José\Mikel-San-Jose", r"Niclas F�llkrug\Niclas-Fullkrug":r"Niclas Füllkrug\Niclas-Fullkrug", r"Niklas S�le\Niklas-Sule":r"Niklas Süle\Niklas-Sule",
                     "Pau L�pez\Pau-Lopez":"Pau López\Pau-Lopez", "Pedro Alcal�\Pedro-Alcala":"Pedro Alcalá\Pedro-Alcala", "Pedro L�pez\Pedro-Lopez":"Pedro López\Pedro-Lopez",
                     "Peter Pekar�k\Peter-Pekarik":"Peter Pekarík\Peter-Pekarik", "Ralf F�hrmann\Ralf-Fahrmann":"Ralf Fährmann\Ralf-Fahrmann", "Ra�l Albentosa\Raul-Albentosa":"Raúl Albentosa\Raul-Albentosa",
                     "Ra�l Bobadilla\Raul-Bobadilla":"Raúl Bobadilla\Raul-Bobadilla", "Ra�l Garc�a\Raul-Garcia":"Raúl García\Raul-Garcia", "Ra�l Navas\Raul-Navas":"Raúl Navas\Raul-Navas",
                     "Rub�n Castro\Ruben-Castro":"Rubén Castro\Ruben-Castro", "Rub�n Duarte\Ruben-Duarte":"Rubén Duarte\Ruben-Duarte", "Rub�n Pardo\Ruben-Pardo":"Rubén Pardo\Ruben-Pardo",
                     "Rub�n Sobrino\Ruben-Sobrino":"Rubén Sobrino\Ruben-Sobrino", "Rub�n\Ruben":"Rubén\Ruben", "R�ben Vezo\Ruben-Vezo":"Rúben Vezo\Ruben-Vezo",
                     "Salif San�\Salif-Sane":"Salif Sané\Salif-Sane", "Samu Garc�a\Samu-Garcia":"Samu García\Samu-Garcia", "Sandro Ram�rez\Sandro-Ramirez":"Sandro Ramírez\Sandro-Ramirez",
                     "Sergio Garc�a\Sergio-Garcia":"Sergio García\Sergio-Garcia", "Sergio Le�n\Sergio-Leon":"Sergio León\Sergio-Leon", "Simon Kj�r\Simon-Kjaer":"Simon Kjær\Simon-Kjaer",
                     "Thiago Alc�ntara\Thiago-Alcantara":"Thiago Alcántara\Thiago-Alcantara", "Thomas M�ller\Thomas-Muller":"Thomas Müller\Thomas-Muller", "Tom�s Pina Isla\Tomas-Pina-Isla":"Tomás Pina Isla\Tomas-Pina-Isla",
                     r"Unai N��ez\Unai-Nunez":r"Unai Núñez\Unai-Nunez", "Vladim�r Darida\Vladimir-Darida":"Vladimír Darida\Vladimir-Darida", "V�ctor Camarasa\Victor-Camarasa":"Víctor Camarasa\Victor-Camarasa",
                     "V�ctor Laguardia\Victor-Laguardia":"Víctor Laguardia\Victor-Laguardia", "V�ctor Ruiz\Victor-Ruiz":"Víctor Ruiz\Victor-Ruiz", "V�ctor S�nchez\Victor-Sanchez":"Víctor Sánchez\Victor-Sanchez",
                     "Willian Jos�\Willian-Jose":"Willian José\Willian-Jose", "Yeray �lvarez\Yeray-Alvarez":"Yeray Álvarez\Yeray-Alvarez", "�lex Granell\Alex-Granell":"Álex Granell\Alex-Granell",
                     "�lvaro Gonz�lez\Alvaro-Gonzalez":"Álvaro González\Alvaro-Gonzalez", "�lvaro Odriozola\Alvaro-Odriozola":"Álvaro Odriozola\Alvaro-Odriozola", "�ngel Rodr�guez\Angel-Rodriguez":"Ángel Rodríguez\Angel-Rodriguez",
                     "�scar Melendo\Oscar-Melendo":"Óscar Melendo\Oscar-Melendo", "�scar de Marcos\Oscar-de-Marcos":"Óscar de Marcos\Oscar-de-Marcos", "�ver Banega\Ever-Banega":"Éver Banega\Ever-Banega"}


for i in range(len(stats)):
  if stats["Player"][i] in player_names_dict:
    stats["Player"][i] = player_names_dict[stats["Player"][i]]

In [ ]:
# save the newer versions of the matches and stats data frames as a csv file

matches.to_csv(source + "matches.csv", index=False)
stats.to_csv(source + "stats.csv", index=False)